In [1]:
import sys, os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
sys.path.append(os.path.normpath(os.getcwd()+os.sep+'src'))
import data_structure as ds
from config import *
from global_definitions import *


case_name = 'ieee4bus'
case_folder = normpath(os.getcwd(),'cases',case_name)
OFOL = normpath(os.getcwd(),'results_experiments',case_name, 'RNN')

print(case_folder)
if not os.path.exists(OFOL):
	os.makedirs(OFOL)

# Time setting
TIME_CONFIG = {
      'tini': datetime(2018, 7, 27, 0, 0, 0),
   'tiniout': datetime(2018, 8, 15, 0, 0, 0),
      'tend': datetime(2018, 8, 21, 23, 59, 54),
        'dt': timedelta(seconds=6),
      'n_rh': 150
   }

# Read case
pdata = ds.Adn()
pdata.read(case_folder, case_name)
if TIME_CONFIG is not None:
  pdata.time_config = TIME_CONFIG

/home/miguel/SEP_Model/cases/ieee4bus


In [27]:
%%capture
import os
import numpy as np
import pandas as pd 
from mpc import OPFac, OPFsto
from datetime import datetime, timedelta
from global_definitions import *
from sklearn.model_selection import train_test_split

class DataSets(object):
  """
    df_input:
      DataFrame
    
    PARAMS_INPUT_ANN:
      Dictionary with the definition of neural network input variables
    
    PARAMS_OUTPUT_ANN:
      Dictionary with the definition of neural network "output" variables


    ratio:
      Ratio of training/validation/test set

    mode: 
      1: TimeSeries
      2: Samples

      if mode is TimeSeries
        hours_show_back:
        hours_predict:
        hours_shift:
        
      elif mode is Samples
    """
  

  def __init__(self, pdata: ds.Adn, 
                     df_input:pd.DataFrame, 
                     PARAMS_INPUT_ANN = PARAMS_INPUT_ANN,
                     PARAMS_OUTPUT_ANN = PARAMS_OUTPUT_ANN,
                     ratio:list or tuple = [7, 2, 1],
                     mode:int = 1,
                     hours_show_back:float = 72.,
                     hours_predict:float = 24.,
                     hours_shift:float = 1.):
    
    self.pdata = pdata
    self.df_input = df_input
    self.df_output = None
    self.mode = mode
    self.ratio = ratio
    self.PARAMS_INPUT_ANN = PARAMS_INPUT_ANN
    self.PARAMS_OUTPUT_ANN = PARAMS_OUTPUT_ANN

    
    self.n_input  = timedelta(hours=hours_show_back)//pdata.time_config['dt']
    self.n_output = timedelta(hours=hours_predict)//pdata.time_config['dt']
    self.n_shift  = timedelta(hours=hours_shift)//pdata.time_config['dt']
    print("Length of the input sample:",self.n_input)
  
  
  def __call__(self):
    self.preprocessing()
    return self.sets()

  @property
  def N_input(self):
    return self.dfx.shape[1]


  def preprocessing(self):
    full_seconds = self.df_input.index.hour*3600+self.df_input.index.minute*60+self.df_input.index.second
    seconds_in_day = timedelta(hours=24).total_seconds()
    self.df_input['sin_time'] = np.sin(2*np.pi*full_seconds/seconds_in_day)
    self.df_input['cos_time'] = np.cos(2*np.pi*full_seconds/seconds_in_day)

    if self.mode==1:
      # *********** run AC_OPF_with_TAPS or another method (replace below?)
      # Calculate hotstart
      sta_opf_hotstart = OPFsto(self.pdata)
      solver_name = sta_opf_hotstart.solver_path.split(os.sep)[-1]
      sta_opf_hotstart.ampl.setOption(solver_name+'_options', 'outlev=0')

      sta_opf_hotstart.ampl.obj
      idx_0 = self.df_input.index[0]
      df_sol_hotstart = sta_opf_hotstart.run(self.df_input.loc[[idx_0], :])

      # Run acopf
      sta_opf = OPFac(self.pdata)
      solver_name = sta_opf.solver_path_sim.split(os.sep)[-1]
      sta_opf.ampl.setOption(solver_name+'_options', 'outlev=0')
      self.df_output = sta_opf.run(self.df_input, df_sol_hotstart)
    
    elif self.mode==2:
      # Calculate hotstart
      sta_opf_hotstart = OPFsto(self.pdata)
      solver_name = sta_opf_hotstart.solver_path.split(os.sep)[-1]
      sta_opf_hotstart.ampl.setOption(solver_name+'_options', 'outlev=0')

      sta_opf_hotstart.ampl.obj
      idx_0 = self.df_input.index[0]
      df_sol_hotstart = sta_opf_hotstart.run(self.df_input.loc[[idx_0], :])

      # Run acopf
      sta_opf = OPFac(self.pdata)
      solver_name = sta_opf.solver_path_sim.split(os.sep)[-1]
      sta_opf.ampl.setOption(solver_name+'_options', 'outlev=0')
      self.df_output = sta_opf.run(self.df_input, df_sol_hotstart)       

  def sets(self):
    try:
      xNames = self.df_input.columns
      yNames = self.df_output.columns

      # search for neural network input variables
      xx = [np.hstack([xNames[[kk in kkk for kkk in xNames]] for kk in self.PARAMS_INPUT_ANN[k]]) for k in self.PARAMS_INPUT_ANN]
      xy = [np.hstack([yNames[[kk in kkk for kkk in yNames]] for kk in self.PARAMS_INPUT_ANN[k]]) for k in self.PARAMS_INPUT_ANN]
      self.dfx = pd.concat([self.df_input[np.hstack(xx)], self.df_output[np.hstack(xy)]], axis=1)

      # search for neural network output variables
      yx = np.hstack([xNames[[kk in kkk for kkk in xNames]] for kk in self.PARAMS_OUTPUT_ANN]) 
      yy = np.hstack([yNames[[kk in kkk for kkk in yNames]] for kk in self.PARAMS_OUTPUT_ANN]) 
      self.dfy = pd.concat([self.df_input[yx], self.df_output[yy]], axis=1)

      
      if self.mode==1:
        samplesX = []
        samplesY = []
        n_sample=0
        
        # Sample generation
        while n_sample <= self.dfx.shape[0]:
          samplesX.append(self.dfx.iloc[n_sample:n_sample+self.n_input].to_numpy)
          samplesY.append(self.dfy.iloc[n_sample+self.n_input:n_sample+self.n_input+self.n_output].to_numpy())
          n_sample+=self.n_shift

        # DataSets
        kp_value = len(samplesX)//sum(self.ratio)
        set1X = samplesX[:self.ratio[0]*kp_value]
        set2X = samplesX[self.ratio[0]*kp_value:sum(self.ratio[:2])*kp_value]
        set3X = samplesX[sum(self.ratio[:2])*kp_value:] 

        set1Y = samplesY[:self.ratio[0]*kp_value]
        set2Y = samplesY[self.ratio[0]*kp_value:sum(self.ratio[:2])*kp_value]
        set3Y = samplesY[sum(self.ratio[:2])*kp_value:] 

      elif self.mode==2:
        # DataSets -> follows the distribution of the input data
        set1X, set2X, set1Y, set2Y = train_test_split(self.dfx, self.dfy, test_size=self.ratio[1]/sum(self.ratio), random_state=0, shuffle=self.dfx)
        set1X, set3X, set1Y, set3Y = train_test_split(set1X, set1Y, test_size=self.ratio[2]/(self.ratio[0]+self.ratio[2]), random_state=0, shuffle=self.dfx)

      return [[set1X, set1Y], [set2X, set2Y], [set3X, set3Y]]
      
    except:
      return [None, None, None]


sets = DataSets(pdata, pdata.df_data.iloc[0:2000], mode=2)
[train, val, test] = sets()

In [28]:
from sklearn.cluster import KMeans

n_sample = 3
n_data = 0
dt = 10


samplesX = []
samplesY = []

# Sample generation
while n_data < sets.dfx.shape[0]:
  dfxx = sets.dfx.iloc[n_data:n_data+dt]
  kmeans = KMeans(n_clusters=n_sample).fit(dfxx)
  df_samples = pd.DataFrame(kmeans.cluster_centers_, columns=dfxx.columns)
  samplesX.append(df_samples)
  n_data+=dt

In [33]:
sets.dfx

,cos_time,sin_time,loadp1,loadp2,loadp3,loadq1,loadq2,loadq3,dgpmax1,dgpmax2,...,ldgp3,V1,V2,V3,dgp1,dgp2,dgp3,dgq1,dgq2,dgq3
time,,,,,,,,,,,,,,,,,,,,,
2018-07-27 00:00:00,1.000000,0.000000,1.026890,0.969397,0.293059,0.497328,0.469488,0.141930,0.0,0.0,...,-0.000756,0.977797,0.963049,0.993044,0.0,0.0,0.0,1.706713,0.333328,-0.333324
2018-07-27 00:00:06,1.000000,0.000436,1.022464,0.965843,0.292055,0.495185,0.467767,0.141444,0.0,0.0,...,-0.000756,0.969106,0.950000,0.997102,0.0,0.0,0.0,0.949583,-0.333280,0.333196
2018-07-27 00:00:12,1.000000,0.000873,1.018039,0.962290,0.291051,0.493042,0.466046,0.140958,0.0,0.0,...,-0.000756,0.978079,0.963461,0.993079,0.0,0.0,0.0,1.706693,0.333307,-0.333279
2018-07-27 00:00:18,0.999999,0.001309,1.013614,0.958736,0.290047,0.490899,0.464325,0.140471,0.0,0.0,...,-0.000756,0.967942,0.950000,0.996896,0.0,0.0,0.0,0.547015,-0.170556,0.293031
2018-07-27 00:00:24,0.999998,0.001745,1.009189,0.955183,0.289043,0.488755,0.462603,0.139985,0.0,0.0,...,-0.000757,0.978362,0.963873,0.993112,0.0,0.0,0.0,1.706705,0.333319,-0.333310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-27 03:19:30,0.644457,0.764640,0.613281,0.653981,0.174985,0.297008,0.316725,0.084744,0.0,0.0,...,-0.000770,0.964693,0.951089,0.995042,0.0,0.0,0.0,-1.706693,-0.333314,-0.333167
2018-07-27 03:19:36,0.644124,0.764921,0.615385,0.655928,0.175588,0.298027,0.317668,0.085036,0.0,0.0,...,-0.000771,0.964619,0.950979,0.999054,0.0,0.0,0.0,-1.706690,-0.333313,0.333030
2018-07-27 03:19:42,0.643790,0.765202,0.617489,0.657876,0.176191,0.299046,0.318611,0.085328,0.0,0.0,...,-0.000772,0.964547,0.950871,0.995026,0.0,0.0,0.0,-1.706578,-0.333245,-0.332479


In [29]:
# df_samples
# samplesX[0]

pd.concat([k.iloc[0] for k in samplesX], axis=1).T

,cos_time,sin_time,loadp1,loadp2,loadp3,loadq1,loadq2,loadq3,dgpmax1,dgpmax2,...,ldgp3,V1,V2,V3,dgp1,dgp2,dgp3,dgq1,dgq2,dgq3
0,0.999998,0.001745,1.009189,0.955183,0.289043,0.488755,0.462603,0.139985,0.0,0.0,...,-0.000757,0.978361,0.963872,0.993113,0.0,0.0,0.0,1.706697,0.333311,-0.333290
0,0.999980,0.006254,0.965467,0.918445,0.278923,0.467580,0.444810,0.135084,0.0,0.0,...,-0.000757,0.979676,0.965812,0.993910,0.0,0.0,0.0,1.694847,0.326981,-0.230860
0,0.999945,0.010363,0.931751,0.886311,0.270642,0.451250,0.429247,0.131073,0.0,0.0,...,-0.000752,0.976378,0.959235,0.993700,0.0,0.0,0.0,1.565846,-0.271041,-0.288200
0,0.999880,0.015489,0.903382,0.852097,0.262721,0.437511,0.412676,0.127237,0.0,0.0,...,-0.000742,0.975155,0.959975,0.997110,0.0,0.0,0.0,0.955435,-0.056069,0.252062
0,0.999821,0.018907,0.893692,0.833971,0.259122,0.432818,0.403898,0.125494,0.0,0.0,...,-0.000734,0.978025,0.962312,0.995164,0.0,0.0,0.0,1.493501,-0.196486,-0.078659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.658689,0.752415,0.533583,0.582126,0.153954,0.258409,0.281924,0.074559,0.0,0.0,...,-0.000746,0.967451,0.955162,0.999417,0.0,0.0,0.0,-1.706707,-0.333326,0.333234
0,0.654740,0.755853,0.550556,0.595610,0.158134,0.266629,0.288455,0.076583,0.0,0.0,...,-0.000750,0.966900,0.954364,0.999344,0.0,0.0,0.0,-1.706678,-0.333301,0.332707
0,0.650774,0.759271,0.573082,0.616168,0.163919,0.277539,0.298411,0.079384,0.0,0.0,...,-0.000757,0.966116,0.953205,0.995229,0.0,0.0,0.0,-1.706679,-0.333300,-0.333042
0,0.647787,0.761820,0.592014,0.634038,0.169019,0.286708,0.307066,0.081855,0.0,0.0,...,-0.000763,0.965446,0.952208,0.995144,0.0,0.0,0.0,-1.706600,-0.333282,-0.332971


,cos_time,sin_time,loadp1,loadp2,loadp3,loadq1,loadq2,loadq3,dgpmax1,dgpmax2,...,ldgp3,V1,V2,V3,dgp1,dgp2,dgp3,dgq1,dgq2,dgq3
0,0.999980,0.006254,0.965467,0.918445,0.278923,0.467580,0.444810,0.135084,0.0,0.0,...,-0.000757,0.979676,0.965812,0.993910,0.0,0.0,0.0,1.694847,0.326981,-0.230860
1,0.999979,0.006254,0.965467,0.918445,0.278923,0.467580,0.444810,0.135084,0.0,0.0,...,-0.000757,0.964059,0.950001,0.997317,0.0,0.0,0.0,-0.820749,0.332818,0.332275
2,0.999976,0.006981,0.958863,0.912515,0.277348,0.464382,0.441938,0.134321,0.0,0.0,...,-0.000756,0.968142,0.950033,0.997339,0.0,0.0,0.0,0.453668,-0.324326,0.331504


In [ ]:
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

Data = {'x': [25,34,22,27,33,33,31,22,35,34,67,54,57,43,50,57,59,52,65,47,49,48,35,33,44,45,38,43,51,46],
        'y': [79,51,53,78,59,74,73,57,69,75,51,32,40,47,53,36,35,58,59,50,25,20,14,12,20,5,29,27,8,7]
       }
  
df = DataFrame(Data,columns=['x','y'])
  
kmeans = KMeans(n_clusters=3).fit(df)
centroids = kmeans.cluster_centers_
# print(centroids)

# plt.scatter(df['x'], df['y'], c= kmeans.labels_.astype(float), s=50, alpha=0.5)
# plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)
# plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l1, l2
from keras.callbacks import History, EarlyStopping, ModelCheckpoint, TerminateOnNaN, CSVLogger   
from keras.layers import Dense, Reshape
from sklearn.preprocessing import StandardScaler

class kerasModel:
  def __init__(self, output_size_G:list or tuple,
                     output_size_D:list or tuple,
                     sets:list,
                     OFOL:str,
                     Loss:str = 'MSE',
                     verbose:bool = False,
                     ):
    
    
    # Sample interpretation
    [train, val, test] = sets
    self.Xtrain = train[0]
    self.Ytrain = train[1]
    self.Xval = val[0]
    self.Yval = val[1]
    self.Xtest = test[0]
    self.Ytest = test[1]

    if verbose:
      print(" "*33+"{:^7s}| {:^7s} | {:^7s}".format("input","target","samples"))
      print("Examples of the training set:   {:^7d} | {:^7d} | {:^7d}".format(self.Xtrain.shape[1], self.Ytrain.shape[1], self.Ytrain.shape[0]))
      print("Examples of the validation set: {:^7d} | {:^7d} | {:^7d}".format(self.Xval.shape[1], self.Yval.shape[1], self.Yval.shape[0]))
      print("Examples of the test set:       {:^7d} | {:^7d} | {:^7d}".format(self.Xtest.shape[1], self.Ytest.shape[1], self.Ytest.shape[0]))
    
    # input data normalization
    self.Xscaler = StandardScaler().fit(pd.concat([self.Xtrain, self.Xval, self.Xtest]))

    # Network properties
    self.N_input = self.Xtrain.shape[1]
    self.output_size_G = output_size_G
    self.output_size_D = output_size_D
    self.Loss = Loss
    
    # project path
    self.OFOL = OFOL

  def __call__(self, dense_layers:list=[100, 70, 40], 
                   actHid:str='tanh', 
                   actOut:str='linear', 
                   actReg:keras.regularizers.L1 or keras.regularizers.L2 = l1(0.15), 
                   seed:int=1234):

    model = self.Create(dense_layers, actHid, actOut, actReg, seed)
    try:
      model = load.model ## añadir opción
    except:
      pass
    return model 


  def Create(self, dense_layers:list=[100, 70, 40], 
                   actHid:str='tanh', 
                   actOut:str='linear', 
                   actReg:keras.regularizers.L1 or keras.regularizers.L2 = l1(0.15), 
                   seed:int=1234):
    self.seed = seed
    np.random.seed(seed), tf.random.set_seed(seed)
    input_dense = keras.Input(shape=(self.N_input,))
    dense = Dense(dense_layers[0], activation=actHid, activity_regularizer=actReg)(input_dense)
    for Nneu in dense_layers[1:]:
      dense = Dense(Nneu, activation=actHid, activity_regularizer=actReg)(dense)
    
    output_convD = Dense(int(self.output_size_D[0]*self.output_size_D[1]), activation=actOut)(dense)
    output_convG = Dense(int(self.output_size_G[0]*self.output_size_G[1]), activation=actOut)(dense)
    
    output_convD = Reshape(self.output_size_D)(output_convD)
    output_convG = Reshape(self.output_size_G)(output_convG)
    model=keras.Model(input_dense, [output_convD, output_convG])
    return model

  
  def TrainingModel(self,
                   epoch:int,
                   lr:float=1e-6,
                   metrics:list = ['MAPE'],
                   dense_layers:list=[100, 70, 40], 
                   actHid:str='tanh', 
                   actOut:str='linear', 
                   actReg:keras.regularizers.L1 or keras.regularizers.L2 = l1(0.15), 
                   seed:int=1234
                   ):

    model = self.Create(dense_layers, actHid, actOut, actReg, seed)
    model.compile(loss=self.CustomLoss, run_eagerly=True, metrics=metrics, optimizer=keras.optimizers.Adam(learning_rate=lr, decay=1e-7))
    Xtrain = self.Xscaler.transform(self.Xtrain)
    Xval = self.Xscaler.transform(self.Xval)
    hist = model.fit(Xtrain, self.Ytrain, validation_data=(Xval, self.Yval))

  def ModelPredict(self, y_pred):
    D, G = y_pred
    ## Modelo de optimización
    # return loss2
    return 0

  
  def CustomLoss(self, y_true, y_pred):
    self.Xtrain

  def Callbacks(self):
    return [History(), 
            TerminateOnNaN(),
            EarlyStopping(patience=5, 
                          monitor="val_loss", 
                          restore_best_weights=True),
            ModelCheckpoint(filepath=self.OFOL+os.sep+'checkpoint'+os.sep+self.seed+os.sep,
                        save_weights_only=False,
                        monitor='val_loss',
                        mode='min',
                        save_best_only=False),
            CSVLogger(self.OFOL+os.sep+'checkpoint'+os.sep+self.seed+os.sep+'history.csv', separator=",", append=True),] 


  def LossFuction(self):
    a=1

  def Metrics(self):
    a=1


output_size_G = [20,10]
output_size_D = [15,10]
ann = kerasModel(output_size_G, output_size_D, [train, val, test], OFOL, verbose=True)
model = ann(seed=1)

tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# es un solo D y G para todo el dia
# - hay que muestrear las variables sistemicas matricialmente
#   donde las columnas pasen a ser la variable y la fila
#   rescate la caracteristica temporal
# - para maximizar la cantidad de datos se puede replicar el 
#   procedimiento empleado en rnn, donde se muestrea una ventana
#   temporal y se aumenta un dt tal que la muestra difiere al caso
#   anterior
# - la ventana de observación puede ser 1 o 2 dias antes 

D,G = model(np.ones([2, 26]))


# para hacer el "control tuning", cuanto se ve hacia atras? 1dia, 1sem, toda la base?